In [20]:
import torch
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import cv2
from torchvision import transforms
from torchvision.transforms import Normalize, Resize, ToTensor
import json






In [21]:
images = torch.load('all_images.pt')
title_vectors = pd.read_csv('title_vectors_imgs_cleaned.csv')
dropped_indexes = []

In [22]:
title_vectors = title_vectors[['price','main_cat','IntLabel','title_stopwords']]

In [23]:
title_vectors

,price,main_cat,IntLabel,title_stopwords
0,42.80,Video_Games,19,dreamgear grx universal wired headset amplifie...
1,4.29,Video_Games,19,jimmy johnsons vr football
2,19.90,Video_Games,19,armorines project swarm
3,9.93,Video_Games,19,largo winch commando sar
4,5.24,Video_Games,19,ch products pro pedals usb flight simulator pe...
...,...,...,...,...
449626,45.00,Beauty,3,labelm intensive repair shampoo conditioner ml
449627,14.99,Beauty,3,manic panic amplified semi permanent color blu...
449628,31.99,Beauty,3,cover n style pink midnight oversized unisex c...
449629,41.47,Beauty,3,walden farms spread pnut choc


In [24]:
labelss = title_vectors['IntLabel']
print(f'labels_size {labelss.size}')
print(f'images_size:  {images.size}')

labels_size 449631
images_size:  449631


In [25]:
def get_size(tensor):
  return tensor.size()


def array_for(x):
    return np.array([get_size(xi) for xi in x])

In [26]:
labels_pre = title_vectors['IntLabel'].size
print(f'labels_size {labels_pre}')
print(f'images_size:  {images.size}')

labels_size 449631
images_size:  449631


In [27]:
sizes = array_for(images)
sizes = sizes.astype('int64')

In [28]:
def remove_images_sizes(min,dim):
  global images
  global title_vectors
  global dropped_indexes
  sizes = array_for(images)
  rare_sizes = np.where(np.unique(sizes[:,dim], return_counts=True)[1] <= min)[0]
  rare_sizes_values = np.unique(sizes[:,dim], return_counts=True)[0][rare_sizes]
  rare_sizes_idx = np.where(np.isin(sizes[:,dim],rare_sizes_values) == True)[0]
  images = np.delete(images,rare_sizes_idx)
  dropped_indexes += rare_sizes_idx.tolist()
  
 
def specific_size_removed(size,dim):
  global images
  global dropped_indexes
  sizes = array_for(images)
  dropped_indexes += np.where(sizes[:,dim] == size)[0].tolist()

  images = images[sizes[:,dim] != size ]


In [29]:
# Removing images sizes with less than 20 occurances in each dimension
print(images.size)
print(len(dropped_indexes))
remove_images_sizes(20,1)
print(images.size)
print(len(dropped_indexes))
remove_images_sizes(80,2)
print(images.size)
print(len(dropped_indexes))

remove_images_sizes(50,1)
print(images.size)
print(len(dropped_indexes))


#removing grayscale images
specific_size_removed(1,0)
print(images.size)
print(len(dropped_indexes))
#!!!!!!!!!!!!!!!!



# updated_sizes = array_for(images)
# dropped_indexes += np.where(updated_sizes[:,1] == 75)[0].tolist()
# images = images[updated_sizes[:,1] != 75 ]


449631
0
449614
17
449340
291
449336
295
449169
462


In [30]:
title_vectors = title_vectors.drop(dropped_indexes)

In [31]:
sizes2 = array_for(images)
np.unique(sizes2[:,1], return_counts=True)

(array([36, 40, 50, 65]), array([ 14324, 338845,  89590,   6410], dtype=int64))

In [32]:
## Next time filter for image size at the extraction stage
b = sizes2[:,1:3]
y = np.array([str(xx) for xx in b])
np.unique(y, return_counts=True)

(array(['[36 36]', '[40 40]', '[50 38]', '[65 50]'], dtype='<U7'),
 array([ 14324, 338845,  89590,   6410], dtype=int64))

In [ ]:
# 4 image tensors for the 4 different images sizes

size_36_36 = images[sizes2[:,1] == 36]
size_40_40 = images[sizes2[:,1] == 40]
size_50_38 = images[sizes2[:,1] == 50]
size_65_50 = images[sizes2[:,1] == 65]
title_vectors_36_36 = title_vectors[sizes2[:,1] == 36]
title_vectors_40_40 = title_vectors[sizes2[:,1] == 40]
title_vectors_50_38 = title_vectors[sizes2[:,1] == 50]
title_vectors_65_50 = title_vectors[sizes2[:,1] == 65]

In [117]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj,torch.Tensor):
            return obj.numpy().tolist()
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [118]:
def dataset(images,title_vectors):


  # # data = torch.zeros(3,images.size,images[0].size()[:,1])
  data = []
  for i,x in enumerate(images):
    title_vectors['title_stopwords'].iloc[i]
    inputs = [x, title_vectors['title_stopwords'].iloc[i], title_vectors['IntLabel'].iloc[i]]
    data.append(inputs)
    # except:
    #   # print(f'img looks like: {x}\n\n\n')
    #   # print(f'size of image is: {x.size()}\n\n\n\n\n')
  with open(f"JSON_data/data{images[0].size()[1]}.json", 'w') as f:
    json.dump(data,f,cls=NpEncoder)


In [124]:
dataset(size_36_36,title_vectors_36_36)
dataset(size_40_40,title_vectors_40_40)
dataset(size_50_38,title_vectors_50_38)
dataset(size_65_50,title_vectors_65_50)

In [120]:
with open(f"JSON_data/data36.json", 'r') as f:
  data = json.load(f)

,0,1,2
0,"[[[251, 251, 251, 251, 251, 252, 252, 252, 252...",tnp products full body plastic and aluminum sn...,19
1,"[[[255, 255, 255, 255, 255, 255, 255, 255, 255...",gam pattern series decals vinyl sticker one co...,19
2,"[[[255, 255, 255, 255, 255, 255, 255, 255, 255...",extremerate custom design chrome silver thumbs...,19
3,"[[[255, 255, 255, 255, 255, 255, 255, 255, 255...",gam pattern series decals vinyl sticker one co...,19
4,"[[[255, 255, 255, 255, 255, 255, 255, 255, 255...",gam pattern series decals vinyl sticker one co...,19
...,...,...,...
14319,"[[[254, 254, 254, 254, 253, 254, 254, 251, 254...",signmission keep out gamer at work sign xbox p...,3
14320,"[[[255, 255, 255, 255, 255, 255, 255, 254, 252...",donna premium collection extra firm mesh wrap,3
14321,"[[[59, 53, 47, 43, 38, 30, 27, 30, 42, 49, 49,...",iphone caseautumnfallreg waterproof shockproof...,3
14322,"[[[255, 255, 255, 255, 255, 255, 255, 255, 255...",alafen mulberry silk super smooth soft eye sle...,3


In [16]:

torch.save(size_36_36,'cleaned_images/images_36_36.pt')
torch.save(size_40_40,'cleaned_images/images_40_40.pt')
torch.save(size_50_38,'cleaned_images/images_50_38.pt')
torch.save(size_65_50,'cleaned_images/images_65_50.pt')
title_vectors_36_36.to_csv('cleaned_titles/labels_36_36.csv')
title_vectors_40_40.to_csv('cleaned_titles/labels_40_40.csv')
title_vectors_50_38.to_csv('cleaned_titles/labels_50_38.csv')
title_vectors_65_50.to_csv('cleaned_titles/labels_65_50.csv')

torch.Size([3, 299, 299])